Apply to autism variants from zhou et al., 2019 https://hb.flatironinstitute.org/asdbrowser/

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mtsplice.utils.vcf_utils import HEADER, COLS, variant_name_convert
from mtsplice import get_data_dir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
DATA = get_data_dir()

In [3]:
gtf = '/data/nasif12/home_if12/chengju/project/recods/Data/shared/Homo_sapiens.GRCh37.75.uniq_exon.gtf'
fasta = '/data/nasif12/home_if12/chengju/project/recods/Data/shared/hg19.nochr.fa'
vcf = DATA + "dis_asd.vcf"

## Apply MMSplice first

In [4]:
from mmsplice.vcf_dataloader import SplicingVCFDataloader
from mmsplice import MMSplice, predict_all_table
from mmsplice.utils import max_varEff

Using TensorFlow backend.
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

In [5]:
dl = SplicingVCFDataloader(gtf, 
                           fasta,
                           vcf,
                           encode = True,
                           split_seq=True, 
                           tissue_specific=True,
                           overhang=(300, 300),
                           tissue_overhang=(300,300))

## Apply MTSplice

In [6]:
model = MMSplice()

Instructions for updating:
Colocations handled automatically by placer.


2020-06-03 22:21:04,845 [WARNING] From /opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2020-06-03 22:21:05,359 [WARNING] From /opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
mmsplice_mtsplice = predict_all_table(model, dl, batch_size=1024)

/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
23it [18:33, 49.03s/it]


In [8]:
oe = pd.read_csv(f"{DATA}/oe_clean.tsv", sep='\t')
oe = oe[['gene', 'oe_lof_upper']]

In [9]:
oe['lof_gene'] = oe.oe_lof_upper < 0.35

In [10]:
autism = pd.read_csv(f"{DATA}/dis_asd.tsv.gz", comment='#', sep='\t', index_col=None)
autism['ID'] = autism.Chr.map(str) + '_' + autism.Pos.map(str) + '_' + autism.Ref.map(str) + '_' + autism.Alt.map(str)

In [11]:
mmsplice_mtsplice['ID'] = mmsplice_mtsplice.ID.apply(variant_name_convert)

In [12]:
mmsplice_mtsplice = pd.merge(mmsplice_mtsplice, autism, on='ID')

In [13]:
mmsplice_mtsplice = pd.merge(mmsplice_mtsplice, oe, left_on="RNA gene symbol", right_on="gene")

In [14]:
mmsplice_mtsplice.to_csv(DATA + "MTSplice_all_tissues.csv")